In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV,LeaveOneOut

import ksource_py as ks

In [2]:
filename = "GF12p_n"
trasl = [118.5497276, -102.897199, 5.0]
rot = [0.0, 0.0, -158*np.pi/180.0]
x2z = True
plist = ks.PList(ks.PTRAC_read, filename, trasl=trasl, rot=rot, switch_x2z=x2z, set_params=False)
print("N =", plist.N)
print("I =", plist.I)
print("err =", np.sqrt(plist.p2))

N = 1
I = 1.0
err = 1.0


In [3]:
N = 1000000
parts,ws = plist.get(N=N)
parts = parts[ws>0]
ws = ws[ws>0]

### Optimizacion de BW

In [4]:
metric = ks.SepVarMetric(ks.Lethargy(10), ks.SurfXY(0), ks.Isotrop())
s = ks.KSource(metric)
s.fit(plist=plist, N=1E5, N_tot=2991283)

Calculating bw ... Done
Optimal bw (silv) = [0.29120679 2.63173281 2.65782241 0.08042671 0.08042671 0.08042671]


In [ ]:
bw_silv = s.optimize_bw(parts, N_tot=2991283, method='silv')
print("bw silverman:")
print(bw_silv)

In [ ]:
bw_mlcv = s.optimize_bw(parts, N_tot=2991283, method='mlcv')
print("bw mlcv:")
print(bw_mlcv)
np.savetxt(filename+"_bw_mlcv.txt", bw_mlcv)

In [5]:
bw_knn = s.optimize_bw(parts, N_tot=1000000, method='knn')
print("bw knn:")
print(bw_knn)
np.savetxt(filename+"_bw_knn.txt", bw_knn)

Calculating KNN ...
batch = 1 / 100
batch = 2 / 100
batch = 3 / 100
batch = 4 / 100
batch = 5 / 100
batch = 6 / 100
batch = 7 / 100
batch = 8 / 100
batch = 9 / 100
batch = 10 / 100
batch = 11 / 100
batch = 12 / 100
batch = 13 / 100
batch = 14 / 100
batch = 15 / 100
batch = 16 / 100
batch = 17 / 100
batch = 18 / 100
batch = 19 / 100
batch = 20 / 100
batch = 21 / 100
batch = 22 / 100
batch = 23 / 100
batch = 24 / 100
batch = 25 / 100
batch = 26 / 100
batch = 27 / 100
batch = 28 / 100
batch = 29 / 100
batch = 30 / 100
batch = 31 / 100
batch = 32 / 100
batch = 33 / 100
batch = 34 / 100
batch = 35 / 100
batch = 36 / 100
batch = 37 / 100
batch = 38 / 100
batch = 39 / 100
batch = 40 / 100
batch = 41 / 100
batch = 42 / 100
batch = 43 / 100
batch = 44 / 100
batch = 45 / 100
batch = 46 / 100
batch = 47 / 100
batch = 48 / 100
batch = 49 / 100
batch = 50 / 100
batch = 51 / 100
batch = 52 / 100
batch = 53 / 100
batch = 54 / 100
batch = 55 / 100
batch = 56 / 100
batch = 57 / 100
batch = 58 / 100
bat